**Source:** [Better Everyday] blog, by [Rob Go], cofounder of NextView.

Fragments of the original article are displayed as block quotations.

[Better Everyday]: https://bettereveryday.vc/how-hard-is-it-to-generate-a-10x-return-on-an-investment-9c1656d6c3af
[Rob Go]: https://bettereveryday.vc/@robgo

> How Hard Is It to Generate a 10X Return on an Investment?
> ===============================

> *Disclaimer: This post gets fairly detailed on venture math. I generally think it’s helpful for founders who take VC money to understand the business of their investors because it impacts their incentives and how they see the world. Those who have no interest in these sorts of topics should move on or go back to watching the price of BTC :)*

> Every time I see blog posts referencing multiples on VC investments, I wonder if writers or readers appreciate how hard it is to generate these kinds of multiples and how multiples should really be calculated.

> Obviously, the way to calculate a return multiple is to divide the amount returned from an investment by the dollars invested. If I invested \$10M in a company and got back \$100M, that’s a 10X return. Seems pretty straightforward, right?

> **The problem arises when multiples are inferred from incomplete data.** It’s quite rare that anybody but the fund manager actually knows the dollars out and dollars returned by a specific investment. And if you have incomplete data, there are usually a number of things that go into calculating or estimating the return. Here are a few other things to think about.

> ## How to Actually Calculate a Investment Return Multiple

> ### Remember each round of financing further dilutes early investors’ money.

> Let’s say a seed investor put in \$1M in a company’s first financing around at a \$10M post-money valuation. 
> The company ultimately sold for \$200M. So, one might infer that the seed investor made \$200M/\$10M= 20X their money.


In [28]:
def multiple(investment, post_money_value, exit_value):
    ownership = (investment / post_money_value)
    money_out = ownership * exit_value
    return money_out / investment

In [35]:
mult = multiple(investment=1.0, post_money_value=10.0, exit_value=200.0)
mult

20.0

In [1]:
def return_on_investment(investment, post_money_value, exit_value):
    mult = multiple(investment, post_money_value, exit_value)
    money_out = mult * investment
    return (money_out - investment) / investment

In [39]:
ROI = return_on_investment(investment=1.0, post_money_value=10.0, exit_value=200.0)
ROI

19.0

The multiple and return on investment are *fractions*, they are unitless. 
You may add a "X" unit (for *times*) after these numbers:
since you have to multiply `multiple` by the investment to get the total money out of the deal.

In [40]:
print("Multiple:", mult, "X") 

Multiple: 20.0 X


These values can a given as a percentage.

In [41]:
print(100.0 * ROI, "%") 

1900.0 %


Both values are independent on the value of the investment:
if the VC had put \$2M instead of \$1M, `multiple` would be the same:

In [42]:
multiple(investment=1.0, post_money_value=10.0, exit_value=200.0)

20.0

However, it depends on the [post-money valuation]: 
if there are initially more investors for example and 
the value after the seed round is \$20M instead of \$10M, the return on investment is lower:

[post-money valuation]: https://en.wikipedia.org/wiki/Post-money_valuation

In [43]:
multiple(investment=1.0, post_money_value=20.0, exit_value=200.0)

10.0

-----

**Question.** What would be the `multiple` for an investment of \$1M if the company had a post-money valuation of \$10M but was sold for \$150M ?

In [44]:
# multiple(investment=?, post_money_value=?, exit_value=?)

> Unfortunately, this is almost never the case.

> In almost any outcome like this, the company goes on to raise more money, usually at higher prices. As new investors come on board, all existing shareholders of the company will have their ownership stake in the business diluted. This also happens when a company expands their option pool.

> Here’s how that plays out for the seed investors.

> Let’s say this theoretical company raises just one more round of financing. It’s a \$10M round at a \$50M post-money valuation. And as part of the round, the option pool of the company is expanded by an additional 10%. 
After this round, the company has their \$200M exit.

> In this example, the flow of the math would be something like this: The seed investor owned 10% after the seed financing. In the next round, the seed investor’s ownership is diluted by 30% (20% due the new financing and 10% due to the expanded option pool).

In [ ]:
def dilution(round_size, post_, OPD): multiple(investment=1.0, post_money_value=20.0, exit_value=200.0)
    return round_size / PPMV + OPD

In [ ]:
dilution(R=10.0, PPMV=50.0, OPD=10.0/100.0)

> So the seed investor’s ownership was actually 7% at exit. 
Thus, their \$1M returns \$14M — a 14X return. 
Pretty great, but meaningfully different from 20X.

In [ ]:
def return_on_investment(I, PMV, E, R, PPMV, OPD):
    ownership = I / PMV
    ownership  = ownership * (1.0 - dilution(R, PPMV, OPD))
    return ownership * E / I

In [ ]:
return_on_investment(I=1.0, PMV=10.0, E=200.0, R=10.0, PPMV=50.0, OPD=0.1)

> Another way to look at it is the effective post-money of the investment. 
At the seed, the investor bought 10% for \$1M. 
But when it was all said and done, the investor actually bought 7% for \$1M. 
It was like they invested in the company at a valuation of \$1M / 7% = \$14.3M.

> But most companies do not have a straight shot to a multi-hundred million dollar exit with just a seed and Series A. 
Most raise multiple rounds, and dilution happens at each round.

> Below is a sample outcome table for a few scenarios.
> ![Fund Multiples & Effective Post-Money](https://cdn-images-1.medium.com/max/600/1*sK0tR1eGTX1t4DiA2Nt0pg.png)

In [48]:
import pandas as pd
import numpy as np

def return_on_investment(dilution, PMV, E):
    return (1.0 - dilution) * E / PMV

dilutions = [30.0, 40.0, 50.0, 60.0, 70.0]

data = []
for dilution in dilutions:
    data.append([dilution, return_on_investment(dilution/100.0, PMV=10.0, E=200.0)])

df = pd.DataFrame(data, columns=["Future Dilution", "Multiple"])
# https://pandas.pydata.org/pandas-docs/stable/style.html

def at_least_10X(val):
    if val >= 10.0:
        return 'background-color: #00ff0030'
    else:
        return 'background-color:'
        
s = df.style.applymap(at_least_10X, subset=["Multiple"])
s


,Future Dilution,Multiple
0,30,14
1,40,12
2,50,10
3,60,8
4,70,6


> What this shows is that the investor’s multiple dramatically changes depending on how many rounds of financing occur after the initial round and the level of dilution of each round.

>I like thinking about this in terms of effective post-money because it creates a more visceral reaction. If this company goes on to raise multiple financing rounds such that new investors and future employees end up with an additional 50%, the seed investor mathematically invested at a $20M post-money valuation. Still good, but not what you think of as seed-stage prices.

>> This is why celebrating big financings isn’t always such a great thing.
>> Apart from the screwed-up incentives that can arise from overcapitalized companies, 
>> each time a company raises money, all prior investors get diluted, 
>> which increases the effective post-money of all the earlier dollars.